# EE-411 Fundamentals of inference and learning, EPFL 
## Exercise Session 3: Classification using k-Nearest Neighbors

In this third set of exercises we will start to see an application of supervised learning, introducing one of the simplest-to-implement supervised machine learning algorithm that can be used to solve both classification and regression problems: **k-Nearest Neighbors (KNN)**.

**What you will learn today:** In this third session, we will construct the kNN algorithm by hand, explaining different ways in which you can compute the distance between elements of a matrix. We will compute the train error and the test error, and we will see how they behave as a function of the parameter $k$ or of the degrees of freedom $N/k$.

In [ ]:
%matplotlib inline
import numpy as np
np.random.seed(1234)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (9, 9)
plt.rcParams["font.size"] = 15

Let us first generate some synthetic data. We will assume that we **know** what is the probability distribution of our datapoints, pay attention that usually we do not know it! 
Our **Generative Model** will be a Gaussian mixture model (GMM): 
* Assume that centroids from class zero and one are geneterated Gaussianly as follows
 $${\vec m}_k^{(0)} \sim \mathcal{N} (\vec{\mu} = [1,0] ; I_2) \qquad {\vec m}_k^{(1)} \sim \mathcal{N} (\vec{\mu} = [0,1] ; I_2) $$ 
* For each class, sample 10 different centroids ${\vec m}_k$
* Sample the actual data from $\mathcal{N} ({\vec m}_k, \frac{1}{5} I_2)$.

Let's sample the centroids and plot them

In [ ]:
# Samples 10 centroids for each class from two different bivariate Normal distributions
centroids_per_class = 10

class0_centroids = [1, 0] + np.random.randn(centroids_per_class, 2)
class1_centroids = [0, 1] + np.random.randn(centroids_per_class, 2)

In [ ]:
# Plot centroids
plt.figure(figsize=((12,5)))
plt.plot(class0_centroids[:, 0], class0_centroids[:, 1], "o", markersize=8)
plt.plot(class1_centroids[:, 0], class1_centroids[:, 1], "x", markersize=8)
plt.title("Centroids")

Once we have the centroids we can sample the actual data

In [ ]:
samples_per_class = 100

# Sample actual data sampling from Normal distributions positioned around the centroids (Pick the centroid of the chosen class uniformly)
class0_labels = np.random.randint(10, size = samples_per_class)
class1_labels = np.random.randint(10, size = samples_per_class)

class0_samples = class0_centroids[class0_labels, :] + np.sqrt(1. / 5) * np.random.randn(samples_per_class, 2)
class1_samples = class1_centroids[class1_labels, :] + np.sqrt(1. / 5) * np.random.randn(samples_per_class, 2)

# Plot data
plt.figure(figsize=((12,5)))
plt.plot(class0_samples[:, 0], class0_samples[:, 1], "o", markersize=8)
plt.plot(class1_samples[:, 0], class1_samples[:, 1], "x", markersize=8)
plt.title("Samples")

This is the problem that we want to solve: we are given these points and we want to find a decision boundary that ensures good generalization.
This is in general our goal: we want to be able to correctly classify new samples, never seen before, once they are given to us.

Let us group the data in a nice way. For binary classification problems like this one, the way data is usually arranged is in

- a matrix $X$ of size $N \times P$, where $N$ is the number of samples and $P$ is the number of features (in our case $N = 200$ and $P = 2$); the nomenclature "feature" is not always clear but usually we mean with it the dimension of the space in which the datapoints leave.
- a label vector $y \in \{0, 1\}^N$ saying to which class each sample belongs to

In [ ]:
# Let's use the stack function to build the feature matrix and vector of labels 
X = np.vstack((class0_samples, class1_samples))
y = np.hstack((np.zeros(samples_per_class), np.ones(samples_per_class)))

print(X)
print(y)

n_samples, n_features = np.shape(X)

Next we compute the distance matrix, a $N \times N$ matrix containing the distance from each sample to all others (do we really need to?)



In [ ]:
def compute_distances(X):
    n_samples = len(X)
    distances = np.zeros((n_samples, n_samples))
    
    # Here goes the algorithm
    for i in range(n_samples):
        for j in range(n_samples):
            distances[i, j] = np.sqrt((X[i, 0] - X[j, 0]) ** 2 + (X[i, 1] - X[j, 1]) ** 2)
    
    return distances

If you know already Python, you can recognize that it is not an efficient implementation. 

We wanto to minimize *for* loops in Python! This is a general rule.

In [ ]:
distances = compute_distances(X)
%timeit compute_distances(X)
print(distances)


In [ ]:
# We can vectorize it!
distances = np.sqrt(np.sum((X[:, np.newaxis] - X) ** 2, 2))
%timeit np.sqrt(np.sum((X[:, np.newaxis] - X) ** 2, 2))
print(distances)

In [ ]:
# Or use something that someone already wrote in C
from scipy.spatial.distance import cdist
distances = cdist(X, X)
%timeit cdist(X, X)
print(distances)

Using the distance matrix we can now write our algorithm!

**Workflow**: 
* Write a function that compute the $k$-nearest neighbor estimate for each point in the training set.
* Use this function to assign a class to each point by majority vote choosing a $k$ of your choice
* Compute the *training error* 


**Tip**: look up for the `np.argpartition` function.



In [ ]:
#  Nice shortcut for looking up to a function is to use **?np.argpartition**
?np.argpartition

Let's write down the function which compute the $k$-nearest neighbour for each datapoint.

In [ ]:
def knn(X, y, k):
    n_samples = len(y)
    distances = cdist(X, X)
    estimate = np.zeros(n_samples)
    
    # For each sample in the training set...
    for i in range(n_samples):
        # Look up k closest samples
        nns = np.argpartition(distances[:, i], k)[:k]
        
        # Assign estimate as a majority vote
        estimate[i] = int(sum(y[nns] == 1) > sum(y[nns] == 0))

    return estimate

Now compute the estimated labels for a given instance $(X,y)$ at a fixed $k$ (e.g. $k=10$).

Once we have the estimated labels we are ready to compute the *training error*.

In [ ]:
est_labels = knn(X, y, 10)
print(est_labels)

# Let us compute the training error
train_error = np.mean(y != est_labels)
print(train_error)

In [ ]:
# Plot data
plt.figure(figsize=((12,5)))
plt.plot(X[y == 0, 0], X[y == 0, 1], "o", markersize=8)
plt.plot(X[y == 1, 0], X[y == 1, 1], "x", markersize=8)

# Draw a red circle around misclassified samples
errors = (y != est_labels)
plt.Figure(figsize=((12,5)))
plt.plot(X[errors, 0], X[errors, 1], "o", color="red", markeredgewidth=3, markerfacecolor="white", markersize=12, alpha=0.5)
#plt.plot(X[errors, 0], X[errors, 1], "o", color="red", mew=3, mfc="white", ms=12, alpha=0.5)

As expected, mistakes happen in regions where the majority of points belong to the other class.

But what happens when we try to classify points outside the training set? 

This step is called the **test step**. This is crucial and we must be very careful while doing this. There are a few different ways of assessing this. For instance, we could have used only part of our data in the training set (usually around 80%), and use the remaining to compute the so-called **test error**.Since in our case however the generative model is known, we might as well just get more samples from it.


$\color{darkred}{Caveat}$: Never use samples used in the training step to assess the test performance!

Please pay attention to the previous lines because during the following lectures we will deal always with $\color{blue}{training}$ and $\color{green}{test}$ error. The difference between them is the same that we encounter when considering $\color{green}{learning}$ and  $\color{blue}{memorizing}$: good learning means low test error **NOT** low training error.

In [ ]:
# Sample test data from the model
testsamples_per_class = 10000

class0_testlabels = np.random.randint(10, size = testsamples_per_class)
class1_testlabels = np.random.randint(10, size = testsamples_per_class)
class0_testsamples = class0_centroids[class0_testlabels, :] + np.sqrt(1. / 5) * np.random.randn(testsamples_per_class, 2)
class1_testsamples = class1_centroids[class1_testlabels, :] + np.sqrt(1. / 5) * np.random.randn(testsamples_per_class, 2)

X_test = np.vstack((class0_testsamples, class1_testsamples))
y_test = np.hstack((np.zeros(testsamples_per_class), np.ones(testsamples_per_class)))

In [ ]:
# Plot test data
plt.figure(figsize=((12,5)))
plt.plot(X_test[y_test == 0, 0], X_test[y_test == 0, 1], "o", ms=8)
plt.plot(X_test[y_test == 1, 0], X_test[y_test == 1, 1], "x", ms=8)

We now need to write a function similar to `knn` that computes the estimates not for the points on the training set, but for points on a new *test* set.


* Write down the function which compute the estimated labels for a point in the test set at fixed $k$, say $k=10$
* Compute with this function the test error

**Tip**: Only distance between test point and training samples matter! Remember the caveat! 

## Evaluated question

* Implement the function `knn_test` which computes the estimated labels for the test set at fixed $k$
* Plot the test and training error as a function of $N/k$
* Do you recognize what is going on? Have we seen this phenomena in the theoretical lectures? Discuss in groups and try to understand what is the **key** concept to grasp from this graph (which apply well beyond the specific case of $k$-nearest neighboour)

In [ ]:
def knn_test(X_train, y_train, X_test, y_test, k):
   ### To complete ###

    return estimate

In [ ]:
est_testlabels = knn_test(X, y, X_test, y_test, 10)

# Let us compute the test error, now
print(f"The test error is {np.mean(y_test != est_testlabels)}")

Great! 

We computed the relevant quantities which allows us to understnad how the algorithm is performing in this learning task. 

Still something is not clear: **How do I choose $k$?**

Indeed when using $k$-nearest neighbors we have to pick a value for $k$ -- it is not clear in principle how to do it! 

In order to understand this better, let us look at how the training and test errors behave as a function of $k$.

In [ ]:
# Convenience functions that compute the training and test errors, given training and test samples
def compute_train_error(X, y, k=1):
    y_hat = knn(X, y, k)
    return np.mean(y != y_hat)
    
def compute_test_error(X_train, y_train, X_test, y_test, k=1):
    y_hat = knn_test(X_train, y_train, X_test, y_test, k)
    return np.mean(y_test != y_hat)

In [ ]:
# Run functions for k belonging to a range of values
ks = np.arange(1, 20)
train_error = []
test_error = []
for (i, k) in enumerate(ks):
    train_error.append(compute_train_error(X, y, k))
    test_error.append(compute_test_error(X, y, X_test, y_test, k))
    print("k = %d; train error = %g, test error = %g" % (k, train_error[-1], test_error[-1]))

# Plot results
plt.plot(ks, train_error, label = "train")
plt.plot(ks, test_error, label = "test")
plt.legend()
plt.xlabel(r"$k$")
plt.ylabel("misclassification error")

It is instructive to use another quantity in the x axis instead of $k$: the number of **degrees of freedom** $N / k$. 

Indeed, the larger the $k$, the smaller the number of effective parameters -- think for instance of the $k = N$ limit, where everyone is assigned the same label.

In [ ]:
### Plot as a function of N/k ### 

In the next lectures we will explore principled way to choose hyperparaemters of the model such as Cross-Validation etc. 

## Bonus section ##

What we have done so far is to give **estimation** of the class to which a datapoint should belong. In general we don't know where our data comes from (see later when we analyze MNIST dataset for example).

Nonetheless we considered a very special case where we knew that the Generative Model was a Gaussian Mixture Model (GMM) around the sampled centroids.

 During theoretical classes we saw that in these cases the best thing I can do is to do Bayesian inference!

Rephrasing mathematically what we have is: $$P(y = 0|x) \propto P(x| y=0) = GMM(\{m_i^{(0)}\}_{i=1}^{10})  \qquad P(y = 1|x) \propto P(x| y=1) = GMM(\{m_i^{(1)}\}_{i=1}^{10}) $$


Let's code toghether a function that assign estimates in a Bayesian way.

In [ ]:
def BayesFunction(class0_centroids,class1_centroids,position):
 diff0 = class0_centroids - position ; diff1 = class1_centroids - position
 exponents0 = np.sum(diff0**2,axis=1) ; exponents1 = np.sum(diff1**2,axis=1)
 p0=sum(np.exp(-2.5*exponents0))
 p1=sum(np.exp(-2.5*exponents1))
 if p0>p1:
  return 0
 else:
  return 1

def ComputeBayesError(class0_centroids,class1_centroids,label,pos):
 frac=0
 for i in range(max(pos.shape)):
  pred = BayesFunction(class0_centroids,class1_centroids,pos[i])
  if pred != label[i]:
     frac=frac+1
 return frac/max(pos.shape)


- Compute the Bayesian error on the training and on the test set
- Plot the Bayesian test error along with the curves of the previous exercise. How does it look like? What can you conclude?

In [ ]:
train_error_bayes = ComputeBayesError(class0_centroids, class1_centroids, y, X)
test_error_bayes = ComputeBayesError(class0_centroids, class1_centroids, y_test, X_test)
print("Bayes train error = %g, Bayes test error = %g" % (train_error_bayes, test_error_bayes))

In [ ]:
 # Plot error as a function of the degrees of freedom
plt.plot(len(y) / np.array(ks), train_error, "-o", label = "train",c='blue')
plt.plot(len(y) / np.array(ks), test_error, "-o", label = "test", c='orange')
plt.axhline(test_error_bayes,linestyle='dashed', label='Bayes test',c='orange')
plt.legend()
plt.xlabel(r"degrees of freedom $N / k$")
plt.ylabel("misclassification error")
#plt.ylim((0.025, 0.15))